## Script Offline Validation


A MODS/SageMaker pipeline includes two part:
- DAG: describing how many steps involved, and how steps are interconnected to each other
- Step: a single unit of functionality, which calls a SageMaker service and usually runs a script

**Cursus** provide an automated solution to transfer PipelineDAG into a SageMaker pipeline by automatically resolved all hidden dependencies, including step-script dependencies, and step-step dependencies.  In other word, Cursus enforce a **contract**:
- "the **producer node** agree that it will provide its output to a *specific location* and the **consumer node** would pick up the output from that *location* as input."
-  Note that the **agreement** is on the **location** of *data transfer* instead of the **content** of *data transfer*.

As a result, the pipeline generated from Cursus can run smoothly *under the condition that* the scripts provided by user gurantee the correct data transfer in terms of its content. For example, the output of training step is given by `model.tar.gz`. Within it, the training script makes sure that it contains the model's pickle file or `.pt` file which can be loaded by the model evaluation script. 

Besides the dependency/connectivity resolution provided by Cursus, we also provide tools to help scientist to verify the functionality and data transfer between scripts locally, before they load it into the pipeline.


In [1]:
import os
import json
import pandas as pd
import pickle
import sys
import boto3
import subprocess
from datetime import datetime
import logging
import argparse

from pathlib import Path

In [2]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [3]:
from pydantic import BaseModel, Field, model_validator, field_validator
from typing import List, Optional, Dict, Any, Union, ClassVar

In [4]:
import sagemaker

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-09-15 22:05:00,744 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


#### Install Session Packages

In [5]:
#!pip uninstall -y rpds-py
#!pip install rpds-py --force-reinstall

In [6]:
#!pip install pyarrow

#### Initiate the Session

In [7]:
from sagemaker import Session

In [8]:
from secure_ai_sandbox_python_lib.session import Session as SaisSession

# Initialize session with team bucket
sais_session = SaisSession(".")

2025-09-15 22:05:01,046 - INFO - CA certs are provided via the AmazonCACerts installation at /home/ec2-user/.local/lib/python3.10/site-packages/amazoncerts
2025-09-15 22:05:01,438 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-09-15 22:05:02,314 - INFO - successfully patched module botocore


#### Add src to the system path

In [9]:
from pathlib import Path
import sys

# Get parent directory of current notebook
project_root = str(Path().absolute().parent.parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"add project root {project_root} into system")

add project root /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src into system


In [10]:
current_dir = Path().absolute()
current_dir

PosixPath('/home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/mods_pipeline_adapter')

## Import Runtime Testers

In [11]:
from buyer_abuse_mods_template.cursus.validation.runtime.runtime_testing import (
    RuntimeTester,
)

In [12]:
current_dir = Path.cwd()

In [13]:
runtime_workspace = current_dir / "runtime_test_workspace"
runtime_workspace.mkdir(parents=True, exist_ok=True)

In [14]:
tester = RuntimeTester("./runtime_test_workspace")
print("✅ Runtime Tester initialized successfully")
print(f"Workspace directory: {tester.workspace_dir}")

✅ Runtime Tester initialized successfully
Workspace directory: runtime_test_workspace


## Prepare the Initial Data

In [15]:
train_data_s3 = "s3://sandboxdependency-abuse-secureaisandboxteamshare-1l77v9am252um/lukexie-AtoZ-xgboost-NA-2-0-0-pipeline-2025-09-09-23-20-52/CradleDataLoading-Training/output/data"
calibration_data_s3 = "s3://sandboxdependency-abuse-secureaisandboxteamshare-1l77v9am252um/lukexie-AtoZ-xgboost-NA-2-0-0-pipeline-2025-09-09-23-20-52/CradleDataLoading-Calibration/output/data"

In [16]:
# Initialize S3 client
s3_client = boto3.client("s3")

2025-09-15 22:05:02,579 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [17]:
def parse_s3_uri(uri):
    uri = uri.replace("s3://", "")
    bucket, *key_parts = uri.split("/")
    key = "/".join(key_parts)
    return bucket, key


def get_s3_files_count(s3_uri):
    s3_client = boto3.client("s3")
    bucket, prefix = parse_s3_uri(s3_uri)

    # Count files in S3
    count = 0
    paginator = s3_client.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        count += len(page.get("Contents", []))
    return count


def download_s3_directory(s3_uri, local_dir):
    s3_client = boto3.client("s3")
    bucket, prefix = parse_s3_uri(s3_uri)

    # Check if local directory has files
    local_dir = Path(local_dir)
    if local_dir.exists():
        local_files_count = len(list(local_dir.glob("*")))
        s3_files_count = get_s3_files_count(s3_uri)

        if local_files_count == s3_files_count:
            print(f"Files already exist in {local_dir}. Skipping download.")
            return
        else:
            print(
                f"Found {local_files_count} files locally vs {s3_files_count} files in S3. Re-downloading..."
            )

    # List and download all objects under the prefix
    paginator = s3_client.get_paginator("list_objects_v2")
    downloaded_count = 0

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            s3_key = obj["Key"]
            file_name = Path(s3_key).name
            local_path = local_dir / file_name

            print(f"Downloading {s3_key} to {local_path}")
            s3_client.download_file(bucket, s3_key, str(local_path))
            downloaded_count += 1

    print(f"Downloaded {downloaded_count} files to {local_dir}")

In [18]:
local_dir = runtime_workspace / "data"
local_dir.mkdir(parents=True, exist_ok=True)

### Download Train data

In [19]:
train_dir = local_dir / "train_data"
train_dir.mkdir(parents=True, exist_ok=True)

calibration_dir = local_dir / "calibration_data"
calibration_dir.mkdir(parents=True, exist_ok=True)

In [20]:
# Download training data
download_s3_directory(train_data_s3, train_dir)

Files already exist in /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/mods_pipeline_adapter/runtime_test_workspace/data/train_data. Skipping download.


In [21]:
download_s3_directory(calibration_data_s3, calibration_dir)

Files already exist in /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/mods_pipeline_adapter/runtime_test_workspace/data/calibration_data. Skipping download.


## Prepare the PipelineDAG

Suppose that we have a planned DAG to be launched to MODS. 

![script_runtime_test_dag](./demo/script_runtime_test_dag.png)

Let us cut the registration and data loading off from the pipeline, since the scripts are fixed and we cannot simulate these steps 


In [22]:
from buyer_abuse_mods_template.cursus.api.dag.base_dag import PipelineDAG

In [23]:
def create_xgboost_pipeline_dag() -> PipelineDAG:
    """
    Create the DAG structure for the XGBoost train-calibrate-evaluate pipeline.

    This is extracted from the _create_pipeline_dag method in the
    XGBoostTrainCalibrateEvaluateE2ETemplate class.

    Returns:
        PipelineDAG: The directed acyclic graph representing the pipeline
    """
    dag = PipelineDAG()

    # Add all nodes - renamed to match configuration names exactly
    # dag.add_node("CradleDataLoading_training")       # Data load for training
    dag.add_node("TabularPreprocessing_training")  # Tabular preprocessing for training
    dag.add_node("XGBoostTraining")  # XGBoost training step
    dag.add_node(
        "ModelCalibration_calibration"
    )  # Model calibration step with calibration variant
    dag.add_node("Package")  # Package step
    # dag.add_node("Registration")                     # MIMS registration step
    dag.add_node("Payload")  # Payload step
    dag.add_node("CradleDataLoading_calibration")  # Data load for calibration
    dag.add_node(
        "TabularPreprocessing_calibration"
    )  # Tabular preprocessing for calibration
    dag.add_node("XGBoostModelEval_calibration")  # Model evaluation step

    # Training flow
    # dag.add_edge("CradleDataLoading_training", "TabularPreprocessing_training")
    dag.add_edge("TabularPreprocessing_training", "XGBoostTraining")

    # Calibration flow
    # dag.add_edge("CradleDataLoading_calibration", "TabularPreprocessing_calibration")

    # Evaluation flow
    dag.add_edge("XGBoostTraining", "XGBoostModelEval_calibration")
    dag.add_edge("TabularPreprocessing_calibration", "XGBoostModelEval_calibration")

    # Model calibration flow - depends on model evaluation
    dag.add_edge("XGBoostModelEval_calibration", "ModelCalibration_calibration")

    # Output flow
    dag.add_edge("ModelCalibration_calibration", "Package")
    dag.add_edge("XGBoostTraining", "Package")  # Raw model is also input to packaging
    dag.add_edge("XGBoostTraining", "Payload")  # Payload test uses the raw model
    # dag.add_edge("Package", "Registration")
    # dag.add_edge("Payload", "Registration")

    logger.info(
        f"Created XGBoost complete E2E DAG with {len(dag.nodes)} nodes and {len(dag.edges)} edges"
    )
    return dag

In [24]:
dag = create_xgboost_pipeline_dag()

2025-09-15 22:05:02,936 - INFO - Added node: TabularPreprocessing_training
2025-09-15 22:05:02,936 - INFO - Added node: XGBoostTraining
2025-09-15 22:05:02,936 - INFO - Added node: ModelCalibration_calibration
2025-09-15 22:05:02,937 - INFO - Added node: Package
2025-09-15 22:05:02,937 - INFO - Added node: Payload
2025-09-15 22:05:02,938 - INFO - Added node: CradleDataLoading_calibration
2025-09-15 22:05:02,938 - INFO - Added node: TabularPreprocessing_calibration
2025-09-15 22:05:02,938 - INFO - Added node: XGBoostModelEval_calibration
2025-09-15 22:05:02,939 - INFO - Added edge: TabularPreprocessing_training -> XGBoostTraining
2025-09-15 22:05:02,939 - INFO - Added edge: XGBoostTraining -> XGBoostModelEval_calibration
2025-09-15 22:05:02,939 - INFO - Added edge: TabularPreprocessing_calibration -> XGBoostModelEval_calibration
2025-09-15 22:05:02,940 - INFO - Added edge: XGBoostModelEval_calibration -> ModelCalibration_calibration
2025-09-15 22:05:02,941 - INFO - Added edge: ModelCali

## Run Single Script Test using the Script Execution Specifications

In [25]:
from buyer_abuse_mods_template.cursus.validation.runtime.runtime_models import (
    ScriptExecutionSpec,
)
from buyer_abuse_mods_template.cursus.validation.runtime.runtime_spec_builder import (
    PipelineTestingSpecBuilder,
)

In [26]:
# Save the specification for reuse
builder = PipelineTestingSpecBuilder(runtime_workspace)

### [REQUIRED] Common Main Interface for each Script

note that each script should have a `main` function with the following interface
```python
def main(
    input_paths: Dict[str, str],
    output_paths: Dict[str, str],
    environ_vars: Dict[str, str],
    job_args: argparse.Namespace,
) -> None:
```
where
- `input_paths` (`Dict[str, str]`): Dictionary of input paths
- `output_paths` (`Dict[str, str]`): Dictionary of output paths
- `environ_vars` (`Dict[str, str]`): Dictionary of environment variables
- `job_args` (`argparse.Namespace`): Command line arguments
e.g.


The script when running withint the container would call main function
```python

if __name__ == "__main__":
    try:
        parser = argparse.ArgumentParser()
        parser.add_argument(
            "--job_type",
            type=str,
            required=True,
            choices=["training", "validation", "testing", "calibration"],
            help="One of ['training','validation','testing','calibration']",
        )
        args = parser.parse_args()

        # Read configuration from environment variables
        LABEL_FIELD = os.environ.get("LABEL_FIELD")
        if not LABEL_FIELD:
            raise RuntimeError("LABEL_FIELD environment variable must be set.")
        TRAIN_RATIO = float(os.environ.get("TRAIN_RATIO", 0.7))
        TEST_VAL_RATIO = float(os.environ.get("TEST_VAL_RATIO", 0.5))

        # Define standard SageMaker paths - use contract-declared paths directly
        INPUT_DATA_DIR = "/opt/ml/processing/input/data"
        OUTPUT_DIR = "/opt/ml/processing/output"

        # Set up logging
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",
        )
        logger = logging.getLogger(__name__)

        # Log key parameters
        logger.info(f"Starting tabular preprocessing with parameters:")
        logger.info(f"  Job Type: {args.job_type}")
        logger.info(f"  Label Field: {LABEL_FIELD}")
        logger.info(f"  Train Ratio: {TRAIN_RATIO}")
        logger.info(f"  Test/Val Ratio: {TEST_VAL_RATIO}")
        logger.info(f"  Input Directory: {INPUT_DATA_DIR}")
        logger.info(f"  Output Directory: {OUTPUT_DIR}")

        # Set up path dictionaries
        input_paths = {"data_input": INPUT_DATA_DIR}

        output_paths = {"data_output": OUTPUT_DIR}

        # Environment variables dictionary
        environ_vars = {
            "LABEL_FIELD": LABEL_FIELD,
            "TRAIN_RATIO": str(TRAIN_RATIO),
            "TEST_VAL_RATIO": str(TEST_VAL_RATIO),
        }

        # Execute the main processing logic
        result = main(
            input_paths=input_paths,
            output_paths=output_paths,
            environ_vars=environ_vars,
            job_args=args,
            logger=logger.info,
        )
```

### Step 1: Tabular Preprocessing Script Execution Spec

In [ ]:
environ_vars = {
    "LABEL_FIELD": "is_abuse",
    "TRAIN_RATIO": "0.7",
    "TEST_VAL_RATIO": "0.5",
}

In [ ]:
# Create a custom script execution specification
tabular_preprocessing_script_spec = ScriptExecutionSpec(
    script_name="tabular_preprocessing",
    step_name="TabularPreprocessing_training",
    input_paths={"data_input": str(train_dir)},
    output_paths={"data_output": str(local_dir / "tabular_preprocessing_training")},
    environ_vars=environ_vars,
    job_args={"job_type": "training"},
)

print("📋 Created custom script execution specification:")
print(f"Script: {tabular_preprocessing_script_spec.script_name}")
print(f"Input paths: {tabular_preprocessing_script_spec.input_paths}")
print(f"Output paths: {tabular_preprocessing_script_spec.output_paths}")
print(f"Environment vars: {tabular_preprocessing_script_spec.environ_vars}")
print(f"Job args: {tabular_preprocessing_script_spec.job_args}")

#### Save the specification for reuse

In [ ]:
saved_path = tabular_preprocessing_script_spec.save_to_file(str(builder.specs_dir))
print(f"💾 Saved specification to: {saved_path}")

#### Get Signatures

In [ ]:
# Test script with custom specification
tabular_preprocessing_main_params = builder.get_script_main_params(
    tabular_preprocessing_script_spec
)

In [ ]:
tabular_preprocessing_main_params

#### Execute Script 

In [ ]:
spec_result = tester.test_script_with_spec(
    tabular_preprocessing_script_spec, tabular_preprocessing_main_params
)

In [ ]:
if spec_result.success:
    print("✅ Script test with custom spec passed!")
else:
    print("❌ Script test with custom spec failed!")
    print(f"Error: {spec_result.error_message}")

#### Calibration Data Processing

In [ ]:
tabular_preprocessing_cali_script_spec = ScriptExecutionSpec(
    script_name="tabular_preprocessing",
    step_name="TabularPreprocessing_calibration",
    input_paths={"data_input": str(calibration_dir)},
    output_paths={"data_output": str(local_dir / "tabular_preprocessing_calibration")},
    environ_vars=environ_vars,
    job_args={"job_type": "calibration"},
)

In [ ]:
tabular_preprocessing_cali_main_params = builder.get_script_main_params(
    tabular_preprocessing_cali_script_spec
)

In [ ]:
cali_spec_result = tester.test_script_with_spec(
    tabular_preprocessing_cali_script_spec, tabular_preprocessing_cali_main_params
)

### Step 2: XGBoost Training Script Execution Spec

In [ ]:
processing_folder = current_dir / "runtime_test_workspace" / "scripts"
processing_folder

In [ ]:
if processing_folder not in sys.path:
    sys.path.insert(0, processing_folder)
    print(f"add project root {processing_folder} into system")

In [ ]:
str(local_dir / "tabular_preprocessing_training")

In [ ]:
trainin_input_paths = {
    "input_path": str(local_dir / "tabular_preprocessing_training"),
    "hyperparameters_s3_uri": str(
        current_dir / "pipeline_config" / "config_NA_xgboost_AtoZ_v2"
    ),
}

In [ ]:
trainin_output_paths = {
    "model_output": str(local_dir / "xgboost_training_model"),
    "evaluation_output": str(local_dir / "xgboost_training_output"),
}

In [ ]:
xgboost_training_script_spec = ScriptExecutionSpec(
    script_name="xgboost_training",
    step_name="XGBoostTraining",
    input_paths=trainin_input_paths,
    output_paths=trainin_output_paths,
    environ_vars={},
    job_args={},
)

print("📋 Created custom script execution specification:")
print(f"Script: {xgboost_training_script_spec.script_name}")
print(f"Input paths: {xgboost_training_script_spec.input_paths}")
print(f"Output paths: {xgboost_training_script_spec.output_paths}")
print(f"Environment vars: {xgboost_training_script_spec.environ_vars}")
print(f"Job args: {xgboost_training_script_spec.job_args}")

In [ ]:
xgboost_training_main_params = builder.get_script_main_params(
    xgboost_training_script_spec
)

In [ ]:
xgboost_training_main_params

In [ ]:
saved_path = xgboost_training_script_spec.save_to_file(str(builder.specs_dir))
print(f"💾 Saved specification to: {saved_path}")

In [ ]:
# Install dependent package locally
#!pip install xgboost

In [ ]:
from runtime_test_workspace.scripts.xgboost_training import main as training_main

In [ ]:
training_main(**xgboost_training_main_params)

### Step 3 Model Evaluation

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(
    local_dir
    / "tabular_preprocessing_calibration"
    / "calibration"
    / "calibration_processed_data.csv",
    index_col=None,
)

In [ ]:
df.columns

In [ ]:
#!pip install "pygam>=0.8.1,<=0.10.1"

In [ ]:
from runtime_test_workspace.scripts.xgboost_model_evaluation import (
    main as evaluation_main,
)

In [ ]:
evaluation_input_paths = {
    "model_input": str(local_dir / "xgboost_training_model"),
    "processed_data": str(local_dir / "tabular_preprocessing_calibration"),
}

In [ ]:
evaluation_output_paths = {
    "eval_output": str(local_dir / "xgboost_model_evaluation_data_output"),
    "metrics_output": str(local_dir / "xgboost_model_evaluation_metric_output"),
}

In [ ]:
evaluation_environ_vars = {"ID_FIELD": "order_id", "LABEL_FIELD": "is_abuse"}

In [ ]:
model_evaluation_script_spec = ScriptExecutionSpec(
    script_name="xgboost_model_evaluation",
    step_name="XGBoostModelEval_calibration",
    input_paths=evaluation_input_paths,
    output_paths=evaluation_output_paths,
    environ_vars=evaluation_environ_vars,
    job_args={"job_type": "calibration"},
)

In [ ]:
model_evaluation_main_params = builder.get_script_main_params(
    model_evaluation_script_spec
)

In [ ]:
model_evaluation_main_params

In [ ]:
saved_path = model_evaluation_script_spec.save_to_file(str(builder.specs_dir))
print(f"💾 Saved specification to: {saved_path}")

In [ ]:
evaluation_main(**model_evaluation_main_params)

### Step 4 Model Calibration

In [ ]:
calibration_input_paths = {
    "evaluation_data": str(local_dir / "xgboost_model_evaluation_data_output")
}

calibration_output_paths = {
    "calibration_output": str(local_dir / "model_calibration_model_output"),
    "metrics_output": str(local_dir / "model_calibration_metrics_output"),
    "calibrated_data": str(local_dir / "model_calibration_data_output"),
}

In [ ]:
calibration_env_vars = {
    "CALIBRATION_METHOD": "gam",
    "LABEL_FIELD": "is_abuse",
    "SCORE_FIELD": "prob_class_1",
    "IS_BINARY": "True",
    "MONOTONIC_CONSTRAINT": "True",
    "GAM_SPLINES": "10",
    "ERROR_THRESHOLD": "0.05",
    "NUM_CLASSES": "2",
    "SCORE_FIELD_PREFIX": "prob_class_",
    "MULTICLASS_CATEGORIES": "[0, 1]",
}

In [ ]:
from runtime_test_workspace.scripts.model_calibration import main as calibration_main

In [ ]:
model_calibration_script_spec = ScriptExecutionSpec(
    script_name="model_calibration",
    step_name="ModelCalibration_calibration",
    input_paths=calibration_input_paths,
    output_paths=calibration_output_paths,
    environ_vars=calibration_env_vars,
    job_args={"job_type": "calibration"},
)

In [ ]:
model_calibration_main_params = builder.get_script_main_params(
    model_calibration_script_spec
)

In [ ]:
model_calibration_main_params

In [ ]:
calibration_main(**model_calibration_main_params)

### Step 5 Packaging

In [ ]:
from runtime_test_workspace.scripts.package import main as package_main

In [ ]:
inference_script_folder = local_dir.parent.parent / "dockers" / "xgboost_atoz"
inference_script_folder

In [ ]:
package_input_paths = {
    "model_input": str(local_dir / "xgboost_training_model"),
    "inference_scripts_input": str(
        local_dir.parent.parent / "dockers" / "xgboost_atoz"
    ),
    "calibration_model": str(local_dir / "model_calibration_model_output"),
}

In [ ]:
package_output_paths = {
    "packaged_model": str(local_dir / "package_output"),
}

In [ ]:
package_script_spec = ScriptExecutionSpec(
    script_name="package",
    step_name="Package",
    input_paths=package_input_paths,
    output_paths=package_output_paths,
    environ_vars={},
    job_args={},
)

In [ ]:
package_main_params = builder.get_script_main_params(package_script_spec)

In [ ]:
package_main(**package_main_params)

In [ ]:
package_model_file_path = local_dir / "package_output" / "model.tar.gz"

In [ ]:
import tarfile

with tarfile.open(package_model_file_path, "r:gz") as tar:
    for member in tar.getmembers():
        print(f"Name: {member.name}")
        print(f"Size: {member.size} bytes")
        print(f"Type: {'Directory' if member.isdir() else 'File'}")
        print("-" * 30)

#### Unpackage Model

In [ ]:
def safe_extract(tar, path):
    def is_within_directory(directory, target):
        abs_directory = os.path.abspath(directory)
        abs_target = os.path.abspath(target)
        prefix = os.path.commonprefix([abs_directory, abs_target])
        return prefix == abs_directory

    for member in tar.getmembers():
        member_path = os.path.join(path, member.name)
        if not is_within_directory(path, member_path):
            raise Exception("Attempted path traversal in tar file")

    tar.extractall(path)

In [27]:
unpackage_folder = local_dir / "inference_model_input"
unpackage_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# Extract safely
with tarfile.open(package_model_file_path, "r:gz") as tar:
    safe_extract(tar, unpackage_folder)

print(f"Files safely extracted to: {unpackage_folder}")

### Step 6 Model Inference

In [28]:
from io import StringIO
import pandas as pd

In [29]:
from runtime_test_workspace.scripts.xgboost_inference import (
    model_fn,
    input_fn,
    predict_fn,
    output_fn,
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [30]:
model_artifacts = model_fn(unpackage_folder)

2025-09-15 22:05:10,471 - INFO - Loading model from /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/mods_pipeline_adapter/runtime_test_workspace/data/inference_model_input
2025-09-15 22:05:10,472 - INFO - Found required file: xgboost_model.bst
2025-09-15 22:05:10,473 - INFO - Found required file: risk_table_map.pkl
2025-09-15 22:05:10,473 - INFO - Found required file: impute_dict.pkl
2025-09-15 22:05:10,474 - INFO - Found required file: feature_columns.txt
2025-09-15 22:05:10,488 - INFO - Loaded 64 ordered feature columns
2025-09-15 22:05:10,490 - INFO - Loading binary calibration model from /home/ec2-user/SageMaker/BuyerAbuseModsTemplate/src/buyer_abuse_mods_template/mods_pipeline_adapter/runtime_test_workspace/data/inference_model_input/calibration/calibration_model.pkl
2025-09-15 22:05:10,504 - INFO - Calibration model loaded successfully


In [31]:
raw_model = model_artifacts["model"]

In [32]:
risk_processors = (model_artifacts["risk_processors"],)
numerical_processor = model_artifacts["numerical_processor"]
feature_importance = model_artifacts["feature_importance"]
config = model_artifacts["config"]
calibrator = model_artifacts["calibrator"]

In [33]:
import pandas as pd

In [34]:
df = pd.read_csv(
    local_dir
    / "tabular_preprocessing_calibration"
    / "calibration"
    / "calibration_processed_data.csv",
    index_col=None,
)

In [35]:
region = "NA"
MODEL_CLASS = "xgboost"
service_name = "AtoZ"

In [36]:
hyparam_filename = (
    f"hyperparameters_{region}_{MODEL_CLASS}.json"  #'hyperparameters.json'
)

In [37]:
from buyer_abuse_mods_template.cursus.steps.hyperparams.hyperparameters_xgboost import (
    XGBoostModelHyperparameters,
)

2025-09-15 22:05:12,266 - pipeline_registry.builder_registry - INFO - Registered builder: BatchTransform -> BatchTransformStepBuilder
2025-09-15 22:05:12,266 - INFO - Registered builder: BatchTransform -> BatchTransformStepBuilder
2025-09-15 22:05:12,268 - pipeline_registry.builder_registry - INFO - Registered builder: CurrencyConversion -> CurrencyConversionStepBuilder
2025-09-15 22:05:12,268 - INFO - Registered builder: CurrencyConversion -> CurrencyConversionStepBuilder
2025-09-15 22:05:12,269 - pipeline_registry.builder_registry - INFO - Registered builder: DummyTraining -> DummyTrainingStepBuilder
2025-09-15 22:05:12,269 - INFO - Registered builder: DummyTraining -> DummyTrainingStepBuilder
2025-09-15 22:05:12,272 - pipeline_registry.builder_registry - INFO - Registered builder: ModelCalibration -> ModelCalibrationStepBuilder
2025-09-15 22:05:12,272 - INFO - Registered builder: ModelCalibration -> ModelCalibrationStepBuilder
2025-09-15 22:05:12,274 - pipeline_registry.builder_regi

In [38]:
config_dir = (
    Path("./pipeline_config") / f"config_{region}_{MODEL_CLASS}_{service_name}_v2"
)

In [39]:
hyparam_path = config_dir / hyparam_filename
with open(hyparam_path, "r") as file:
    hyperparam_dict = json.load(file)

In [40]:
hyperparam_dict

{'alpha_xgb': 0.0,
 'base_score': None,
 'batch_size': 2,
 'booster': 'gbtree',
 'cat_field_list': ['PAYMETH',
  'claim_reason',
  'claimantInfo_status',
  'shipments_status'],
 'categorical_features_to_encode': [],
 'class_weights': [1.0, 1.0],
 'colsample_bylevel': 1.0,
 'colsample_bynode': 1.0,
 'colsample_bytree': 1.0,
 'device': -1,
 'early_stopping_rounds': None,
 'eta': 0.3,
 'full_field_list': ['Abuse.abuse_fap_action_by_customer_inline_transform_na.n_claims_solicit_count_last_365_days',
  'Abuse.abuse_fap_action_by_customer_inline_transform_na.n_claims_warn_count_last_365_days',
  'Abuse.abuse_fap_action_by_customer_inline_transform_na.n_concession_solicit_count_last_365_days',
  'Abuse.abuse_fap_action_by_customer_inline_transform_na.n_concession_warn_count_last_365_days',
  'Abuse.bsm_stats_for_evaluated_mfn_concessions_by_customer_na.n_max_buyer_order_message_time_gap',
  'Abuse.bsm_stats_for_evaluated_mfn_concessions_by_customer_na.n_max_order_message_time_gap',
  'Abuse.b

In [41]:
tab_field_list = hyperparam_dict["tab_field_list"]
cat_field_list = hyperparam_dict["cat_field_list"]

In [42]:
input_fields = tab_field_list + cat_field_list

In [43]:
df_sample = df.loc[:, input_fields].sample(10)

In [44]:
df_sample

,Abuse.abuse_fap_action_by_customer_inline_transform_na.n_claims_solicit_count_last_365_days,Abuse.abuse_fap_action_by_customer_inline_transform_na.n_claims_warn_count_last_365_days,Abuse.abuse_fap_action_by_customer_inline_transform_na.n_concession_solicit_count_last_365_days,Abuse.abuse_fap_action_by_customer_inline_transform_na.n_concession_warn_count_last_365_days,Abuse.bsm_stats_for_evaluated_mfn_concessions_by_customer_na.n_max_buyer_order_message_time_gap,Abuse.bsm_stats_for_evaluated_mfn_concessions_by_customer_na.n_max_order_message_time_gap,Abuse.bsm_stats_for_evaluated_mfn_concessions_by_customer_na.n_max_seller_order_message_time_gap,Abuse.bsm_stats_for_evaluated_mfn_concessions_by_customer_na.n_message_count_with_diff_topic_si,Abuse.bsm_stats_for_evaluated_mfn_concessions_by_customer_na.n_message_count_with_notr_topic_si,Abuse.bsm_stats_for_evaluated_mfn_concessions_by_customer_na.n_message_count_with_return_keywords_si,...,Abuse.shiptrack_flag_by_order.n_any_returned,COMP_DAYOB,claimAmount_value,claimantInfo_allClaimCount365day,claimantInfo_lifetimeClaimCount,claimantInfo_pendingClaimCount,PAYMETH,claim_reason,claimantInfo_status,shipments_status
37012,0,0,0,0,0,0,0,0.0,0.000000,0.000000,...,0,1936.716594,579.98,3,10,1,CC,NOTR,Normal,IN_TRANSIT
12308,0,0,0,0,2759489,2784632,2784632,0.0,1.000000,0.000000,...,0,2190.818464,5.26,1,1,1,CC,NOTR,Normal,IN_TRANSIT
34592,0,0,0,0,1995145,1997612,1997612,0.0,1.000000,0.000000,...,0,1505.455934,38.10,1,1,1,CC,NOTR,Normal,DELIVERED
15346,0,0,0,0,0,0,0,0.0,0.000000,0.000000,...,0,2778.261405,24.98,1,1,1,BA,NOTR,Normal,UNKNOWN
35177,0,0,0,0,1498645,1555209,1555209,0.0,0.125000,0.875000,...,0,6776.185582,40.33,1,7,1,CC,NOTR,Normal,DELIVERED
13392,0,0,0,0,4032723,4088376,4088376,0.0,0.571429,0.285714,...,0,801.979859,2.51,1,8,1,GC,NOTR,Normal,IN_TRANSIT
13288,0,0,0,0,0,0,0,0.0,0.000000,0.000000,...,0,1467.013600,31.58,1,1,1,CC,NOTR,Normal,IN_TRANSIT
7418,0,0,0,0,0,0,0,0.0,0.000000,0.000000,...,0,2684.549479,719.44,1,1,1,CC,NOTR,Normal,IN_TRANSIT
21621,0,0,0,0,1327014,1346665,1346665,0.0,1.000000,0.000000,...,0,2558.861170,47.44,1,1,1,CC,NOTR,Normal,DELIVERED
43724,0,0,0,0,825031,832421,832421,0.0,0.500000,1.000000,...,0,4824.446159,108.36,1,1,1,CC,NOTR,Normal,RETURNING


In [45]:
payload = df_sample.iloc[[0]].to_csv(index=False, header=False).strip().encode("utf-8")

In [46]:
payload

b'0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,153,172591.87,264,80,153919.2,158,0.0,0,0.0,0,869.09,1,0.0,0,869.09,1,0,0.0,0,1,869.09,1,0.0,0.005646,4,6876.57,8,0.044676,66,0,0,0,0,0,0,1936.716594,579.98,3,10,1,CC,NOTR,Normal,IN_TRANSIT'

In [47]:
CONTENT_TYPE_CSV = "text/csv"
CONTENT_TYPE_JSON = "application/json"
CONTENT_TYPE_PARQUET = "application/x-parquet"

In [48]:
processed_input = input_fn(payload, CONTENT_TYPE_CSV)

2025-09-15 22:05:17,115 - INFO - Received request with Content-Type: text/csv
2025-09-15 22:05:17,116 - INFO - Processing content type: text/csv
2025-09-15 22:05:17,119 - INFO - Successfully parsed CSV into DataFrame. Shape: (1, 64)


In [49]:
processed_input

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,1936.716594,579.98,3,10,1,CC,NOTR,Normal,IN_TRANSIT


In [50]:
predictions = predict_fn(processed_input, model_artifacts)

2025-09-15 22:05:19,055 - INFO - Applied calibration to predictions


In [51]:
predictions

{'raw_predictions': array([[9.9999434e-01, 5.6872113e-06]], dtype=float32),
 'calibrated_predictions': array([[0.9958394 , 0.00416057]], dtype=float32)}

In [52]:
response = output_fn(predictions, CONTENT_TYPE_JSON)

2025-09-15 22:07:30,502 - INFO - Received prediction output of type: <class 'dict'> for accept type: application/json


In [53]:
response

('{"predictions": [{"legacy-score": "5.687211341864895e-06", "calibrated-score": "0.004160573706030846", "custom-output-label": "class-0"}]}',
 'application/json')

In [57]:
input_fn_results = []
predict_fn_results = []
output_fn_results = []
end_to_end_results = []

In [58]:
for index, row in df_sample.iterrows():
    # Convert single row to DataFrame and then to CSV bytestring
    row_df = pd.DataFrame([row])  # Convert row to single-row DataFrame
    payload = row_df.to_csv(index=False, header=False).strip().encode("utf-8")

    processed_input = input_fn(payload, CONTENT_TYPE_CSV)
    input_fn_results.append(
        {"success": True, "sample": index, "response": processed_input}
    )

    predictions = predict_fn(processed_input, model_artifacts)
    predict_fn_results.append(
        {"success": True, "sample": index, "response": predictions}
    )

    response = output_fn(predictions, CONTENT_TYPE_JSON)
    output_fn_results.append({"success": True, "sample": index, "response": response})

    end_to_end_results.append({"success": True, "sample": index})

2025-09-15 22:13:07,614 - INFO - Received request with Content-Type: text/csv
2025-09-15 22:13:07,615 - INFO - Processing content type: text/csv
2025-09-15 22:13:07,618 - INFO - Successfully parsed CSV into DataFrame. Shape: (1, 64)
2025-09-15 22:13:07,701 - INFO - Applied calibration to predictions
2025-09-15 22:13:07,705 - INFO - Received prediction output of type: <class 'dict'> for accept type: application/json
2025-09-15 22:13:07,708 - INFO - Received request with Content-Type: text/csv
2025-09-15 22:13:07,708 - INFO - Processing content type: text/csv
2025-09-15 22:13:07,711 - INFO - Successfully parsed CSV into DataFrame. Shape: (1, 64)
2025-09-15 22:13:07,787 - INFO - Applied calibration to predictions
2025-09-15 22:13:07,791 - INFO - Received prediction output of type: <class 'dict'> for accept type: application/json
2025-09-15 22:13:07,794 - INFO - Received request with Content-Type: text/csv
2025-09-15 22:13:07,794 - INFO - Processing content type: text/csv
2025-09-15 22:13:

In [60]:
output_fn_results

[{'success': True,
  'sample': 37012,
  'response': ('{"predictions": [{"legacy-score": "5.687211341864895e-06", "calibrated-score": "0.004160573706030846", "custom-output-label": "class-0"}]}',
   'application/json')},
 {'success': True,
  'sample': 12308,
  'response': ('{"predictions": [{"legacy-score": "1.0827107388422519e-07", "calibrated-score": "0.004159283824265003", "custom-output-label": "class-0"}]}',
   'application/json')},
 {'success': True,
  'sample': 34592,
  'response': ('{"predictions": [{"legacy-score": "3.2136347272171406e-06", "calibrated-score": "0.004160001873970032", "custom-output-label": "class-0"}]}',
   'application/json')},
 {'success': True,
  'sample': 15346,
  'response': ('{"predictions": [{"legacy-score": "5.591953140537953e-06", "calibrated-score": "0.004160551354289055", "custom-output-label": "class-0"}]}',
   'application/json')},
 {'success': True,
  'sample': 35177,
  'response': ('{"predictions": [{"legacy-score": "1.8092830487148603e-06", "cal

## Run Data Compatiability Test between Two Adjacent Steps